In [1]:
%%time
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import geopandas as gpd
import matplotlib.pyplot as plt
import numpy as np
from shapely.ops import transform

from shapely.geometry import Point, Polygon
import datetime

from io import StringIO
from pathlib import Path, PureWindowsPath
import os
import sys
from functools import partial

# Import custom functions from `scripts` folder
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from scripts.clean_tweets import geometrize_tweets, convert_shapefile_crs, find_frequencies
from scripts.home_location import assign_home_location
from scripts.summary_stats import summary_stats

data_path = Path("C:/Users/emman/Box/Spring 2020/Displacement Studio/Datasets/")

Wall time: 617 ms


In [2]:
def filter_and_home_assign(data, shapefile, lower, upper):
    """
    Find subset of data to analyze based on lower/upper bounds on tweets/user, 
    then add home locations.
    
    Parameters
    ----------
    data : pd.DataFrame or gpd.GeoDataFrame
        DataFrame containing tweets; must contain column `u_id` for user id
    
    shapefile : gpd.GeoDataFrame
        Shapefile for tracts in tweets location.
        Must be in WGS84 (epsg:4326) format (to align with tweet lat/lon).
    
    lower, upper : int or float
        Lower (inclusive) and upper (exclusive) bounds on tweets/user.
        
    Returns
    -------
    filtered : gpd.GeoDataFrame
        Collection of tweets filtered by lower/upper and with tweet geographies.
        Contains new columns:
            - 'timestamp' : pd.Timestamp for time of tweet published
            - 'date' : date (int) of tweet published
            - 'hour' : 24-hour (int) of tweet published
            - 'home' : name of home location codigo (some may be NaN)
            - 'is_home' : boolean indicating whether tweet was made at home
    """
    # Filter based on lower and upper bound on tweets/user
    filtered = data.groupby('u_id').filter(lambda group: (len(group) >= lower) & (len(group) < upper))
    
    # Geometrize tweets (inplace) based on lat/lon
    filtered = geometrize_tweets(filtered)
    
    # Spatial join with tracts
    filtered = gpd.sjoin(filtered, shapefile, how='left', op='intersects')
    
    # Add datetime
    filtered['timestamp'] = pd.to_datetime(filtered['created_at'] // 1000, unit='s')
    filtered['date'] = filtered['timestamp'].dt.date
    filtered['hour'] = filtered['timestamp'].dt.hour
    
    # Add home location
    filtered['home'] = assign_home_location(filtered, SA2='SA2_MAIN16')
    filtered['is_home'] = filtered['SA2_MAIN16'] == filtered['home']
    
    return filtered

In [3]:
tweets_2015 = pd.read_csv(data_path/'AUST_TWT_2015.csv')

In [4]:
au_shape_sa2 = gpd.read_file('C:\\Users\\emman\\box\\Spring 2020\\Displacement Studio\\Shared 228 Sydney Folder\\SA2 Shapefiles\\SA2_2016_AUST.dbf')
# Crikey, our area is too big
au_shape_sa2.plot()
# We also need to change the crs 
au_shape_sa2.crs = {'init': 'epsg:4326'}
# have the shapefile only have areas that are in the state of New South Wales, and furthermore, in Greater Sydney
au_shape_sa2 = au_shape_sa2[au_shape_sa2.STE_NAME16 == 'New South Wales']
au_shape_sa2 = au_shape_sa2[au_shape_sa2.GCC_NAME16 == 'Greater Sydney']

In [5]:
tweets_2015.head()

,Unnamed: 0,id,created_at,text,u_id,u_name,u_created_at,u_description,u_followers_count,type,c_country,u_screen_name,u_friends_count,location.lon,location.lat,geometry,timestamp,date
0,2772582,550447293917585408,1.420072e+12,An ok view. #fastforward #hyperlapse\n\nHAPPY...,105942338,NeilTea,1.263776e+12,Chief Sound Chef. Made in Bondi.,148,llp,AUS,Neil_Tea,767,151.215190,-33.879437,POINT (151.21519 -33.879437),2015-01-01 00:23:56,2015-01-01 00:00:00
1,2772583,550449135875149826,1.420072e+12,in 2015 im going to get a tattoo on my ass of ...,216618670,Cody Ko,1.289980e+12,humanitarian/playboy. codyko on IG and snap.,92175,llp,AUS,codyk,233,151.282489,-33.890671,POINT (151.282489 -33.890671),2015-01-01 00:31:15,2015-01-01 00:00:00
2,2772584,550449952472576002,1.420072e+12,@TobyCal were you hoping for the 3Ds- broken d...,46484657,Shana Allen,1.244753e+12,SF native turned Sydneysider. Classically trai...,769,llp,AUS,shanaallen,1004,151.206602,-33.863197,POINT (151.206602 -33.863197),2015-01-01 00:34:30,2015-01-01 00:00:00
3,2772586,550454246009618432,1.420073e+12,@KianAndJc DOES THE TOUr inCLUDE AUSTRALIA,514183205,"loz,!⭐️// FEB 7!!",1.330856e+12,"roses are red, violets are blue, jade thirlwal...",3639,llp,AUS,penguinthirls,3445,150.873429,-33.921123,POINT (150.873429 -33.921123),2015-01-01 00:51:33,2015-01-01 00:00:00
4,2772587,550472812000210944,1.420078e+12,Sydney #TurnUp,17925031,Danny Brown,1.228584e+12,"Mgmt: Goliath Artists, Paul Rosenberg/ Dart Pa...",299027,llp,AUS,xdannyxbrownx,29005,151.211378,-33.921803,POINT (151.211378 -33.921803),2015-01-01 02:05:20,2015-01-01 00:00:00


In [6]:
pct_50, pct_99 = summary_stats(tweets_2015)

979761 total tweets
40845 unique users

Median number of tweets/user: 3.0 tweets
99th percentile of tweets/user: 303.0 tweets


In [7]:
twt2015 = filter_and_home_assign(tweets_2015, au_shape_sa2, pct_50, pct_99)

C:\Users\emman\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\plotting.py:487: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


In [8]:
### Plot all 2014 tweets ###
# Green tweets: Joined to an SA2
# Orange tweets: No SA2 Assigned
# Gray: Australia shapefile

fig, ax = plt.subplots(figsize=(20, 14))
au_shape_sa2['geometry'].plot(ax=ax, color='gray')
sample = twt2015.sample(frac=1.00, random_state=42)
sample[~sample['SA2_MAIN16'].isnull()].plot(ax=ax, marker='o', color='green', alpha=0.01, label='Green: Within Greater Sydney')
sample[sample['SA2_MAIN16'].isnull()].plot(ax=ax, marker='o', color='orange', alpha=0.01, label='Orange: Outside Greater Sydney')
plt.legend()
plt.title('Sydney tweets, 2015: Syndey vs. outside Sydney');

From this, we see that we can use [~autwt_2014['SA2_5DIG16'].isnull()] as a filter to select just the tweets in Syndey, which is our area of interest. For all following analyses, we will restrict the population to just the tweets that fall inside Sydney depicted by the shapefile.

Our first summary statistic computes the ratio of tweets that have a home assignment.

In [9]:
num_home_assigned_tweets = (~twt2015.loc[~twt2015['SA2_MAIN16'].isnull(), 'home'].isnull()).sum()
n_tweets = len(twt2015[~twt2015['SA2_MAIN16'].isnull()])

print("{}/{} = {:.2%} of tweets have a home assignment.".format(
    num_home_assigned_tweets, n_tweets, num_home_assigned_tweets / n_tweets
))

307454/554166 = 55.48% of tweets have a home assignment.


Next, we are interested in the ratio of users that have a home assignment.

In [10]:
num_users_with_home = (~twt2015.loc[~twt2015['SA2_MAIN16'].isnull()].groupby('u_id')['home'].first().isnull()).sum()
n_users = len(twt2015[~twt2015['SA2_MAIN16'].isnull()].groupby('u_id'))

print("{}/{} = {:.2%} of users have a home assignment.".format(
    num_users_with_home, n_users, num_users_with_home / n_users
))

3529/22935 = 15.39% of users have a home assignment.


Lastly, we want to know the ratio of tweets that were made from home (i.e. SA2_5DIG16 and home are equal).

In [11]:
twt2015.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 554166 entries, 1 to 979758
Data columns (total 34 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   Unnamed: 0         554166 non-null  int64         
 1   id                 554166 non-null  int64         
 2   created_at         554166 non-null  float64       
 3   text               554166 non-null  object        
 4   u_id               554166 non-null  int64         
 5   u_name             554166 non-null  object        
 6   u_created_at       554166 non-null  float64       
 7   u_description      554166 non-null  object        
 8   u_followers_count  554166 non-null  int64         
 9   type               554166 non-null  object        
 10  c_country          554166 non-null  object        
 11  u_screen_name      554166 non-null  object        
 12  u_friends_count    554166 non-null  int64         
 13  location.lon       554166 non-null  

In [12]:
num_tweets_from_home = twt2015.loc[~twt2015['SA2_MAIN16'].isnull(), 'is_home'].sum()

print("{}/{} = {:.2%} of tweets are posted from the home tract.".format(
    num_tweets_from_home, n_tweets, num_tweets_from_home / n_tweets
))

0/554166 = 0.00% of tweets are posted from the home tract.


C:\Users\emman\AppData\Local\Continuum\anaconda3\lib\site-packages\geopandas\plotting.py:487: UserWarning: The GeoDataFrame you are attempting to plot is empty. Nothing has been displayed.
  UserWarning,


In [13]:
### Plot a 5% sample of 2014 tweets ###
# Blue tweets: Resident
# Red tweets: Non-resident
# Gray: Sydney shapefile

fig, ax = plt.subplots(figsize=(20, 14))

au_shape_sa2['geometry'].plot(ax=ax, color='gray')

sample = twt2015[~twt2015['SA2_MAIN16'].isnull()].sample(frac=0.05, random_state=42)

sample[~sample['is_home']].plot(ax=ax, marker='o', color='red', alpha=0.01, label='Red: Non-Resident')

sample[sample['is_home']].plot(ax=ax, marker='o', color='blue', alpha=0.01, label='Blue: Resident')

plt.legend()
plt.title('Sydney tweets, 2014: Resident vs. Non-resident');

Our analysis concludes with a visual representation of this last information; namely, how are the tweets spread out across the entire metropolitan area of Buenos Aires with respect to residence/nonresidence?

# Saving Output

For future reference, we save an output CSV of all the tweets, spatially joined to the Sydney shapefiles. There are two additional columns:

    home: NaN (missing value) if the tweet user has no home location, else the home tract ('SA2_MAIN16')
    is_home: an indicator (True/False) determining whether home == SA2_MAIN16


In [15]:
#twt2015.to_csv(data_path/'au_twt2015_joined.csv.gz', index=False, compression='gzip')